# Predict velocity values using deep learning

This notebook trains and uses an LSTM to predict velocity values. Modified from the code in [Encord's](https://encord.com/blog/time-series-predictions-with-recurrent-neural-networks/) post. An LSTM uses a memory cell to store information that the model uses to predict future values. Input, forget, and output gates control what is stored in, removed, and output from the LSTM. The LSTM's memory makes it ideal for predicting time series values; the lookback must be modified to encapsulate enough days to predict but not too many for overfitting.

---

### 1. Load packages and open dataset

Choose a point (`upstream`, `middle`, `terminus`).

In [19]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

In [20]:
point_name = 'middle' # 'upstream', 'middle', or 'terminus'
input_file='../data/ai_ready/' + point_name + '_timeseries.csv'

# load the dataset, drop the series column
df = pd.read_csv(input_file)

df.drop(columns="series", inplace=True)
dataset=df['velocity'].values.reshape(-1, 1)
dataset.shape

(364, 1)

### 2. Preprocessing

Here, we convert the values into a matrix, normalize, and split into train and test data. Normalization ensures that predictions are scaled properly so that one data point does not dominate training or predictions. We split into 80/20 train/test data and use a look back of 15, meaning that the model will consider the previous 15 values to predict the next value.

In [21]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	print(len(dataX), len(dataY))
	return np.array(dataX), np.array(dataY)

In [22]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset.shape

(364, 1)

In [23]:
# split into train and test sets, 70% test data, 30% training data
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
train.shape, test.shape

((254, 1), (110, 1))

In [24]:
# reshape into X=t and Y=t+1, timestep 5
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = trainX[:-1]
trainY = trainY[:-1]
trainX.shape, trainY.shape, testX.shape, testY.shape

248 248
104 104


((247, 5), (247,), (104, 5), (104,))

### 3. Train and test

Create the model and train for 5000 epochs. Save it to the `models` folder.

In [25]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

trainX.shape, trainY.shape, testX.shape, testY.shape

((247, 5, 1), (247,), (104, 5, 1), (104,))

In [26]:
# create and fit the LSTM network, optimizer=adam, 50 neurons, 2 LSTM layers
lstm_model = Sequential()
lstm_model.add(LSTM(100, return_sequences=True, input_shape=(look_back, 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(100, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mse', optimizer='adam')
lstm_model.fit(trainX, trainY, epochs=500, batch_size=32, validation_data=(testX, testY), verbose=2)

Epoch 1/500


/Users/cjense/miniconda3/envs/mlgeo-jensencc/lib/python3.10/site-packages/keras/layers/rnn/rnn.py:205: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 - 2s - 228ms/step - loss: 0.2082 - val_loss: 0.0871
Epoch 2/500
8/8 - 0s - 8ms/step - loss: 0.0488 - val_loss: 0.0223
Epoch 3/500
8/8 - 0s - 8ms/step - loss: 0.0284 - val_loss: 0.0407
Epoch 4/500
8/8 - 0s - 9ms/step - loss: 0.0274 - val_loss: 0.0154
Epoch 5/500
8/8 - 0s - 12ms/step - loss: 0.0234 - val_loss: 0.0115
Epoch 6/500
8/8 - 0s - 10ms/step - loss: 0.0198 - val_loss: 0.0190
Epoch 7/500
8/8 - 0s - 9ms/step - loss: 0.0205 - val_loss: 0.0141
Epoch 8/500
8/8 - 0s - 9ms/step - loss: 0.0205 - val_loss: 0.0119
Epoch 9/500
8/8 - 0s - 8ms/step - loss: 0.0204 - val_loss: 0.0177
Epoch 10/500
8/8 - 0s - 8ms/step - loss: 0.0211 - val_loss: 0.0125
Epoch 11/500
8/8 - 0s - 8ms/step - loss: 0.0222 - val_loss: 0.0158
Epoch 12/500
8/8 - 0s - 9ms/step - loss: 0.0205 - val_loss: 0.0146
Epoch 13/500
8/8 - 0s - 8ms/step - loss: 0.0197 - val_loss: 0.0137
Epoch 14/500
8/8 - 0s - 9ms/step - loss: 0.0216 - val_loss: 0.0156
Epoch 15/500
8/8 - 0s - 9ms/step - loss: 0.0200 - val_loss: 0.0131
Epoch 16/500

In [27]:
# make predictions
trainPredict_lstm = lstm_model.predict(trainX)
testPredict_lstm = lstm_model.predict(testX)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [28]:
lstm_model.save('../models/' + point_name + 'lstm_model.keras')

In [29]:
# invert predictions
trainPredict_lstm = scaler.inverse_transform(trainPredict_lstm)
trainY = scaler.inverse_transform([trainY])
testPredict_lstm = scaler.inverse_transform(testPredict_lstm)
testY = scaler.inverse_transform([testY])

### 4. Analyze and plot predictions

Calculate RMSE for train and test, shift for plotting, and plot using Plotly. Note that the first 15 timesteps of both train and test are not accounted for in the prediction data since they were used to train the model.

In [30]:
# calculate root mean squared error
trainScore_lstm = math.sqrt(mean_squared_error(trainY[0], trainPredict_lstm[:,0]))
print('Train Score: %.2f RMSE' % (trainScore_lstm))
testScore_lstm = math.sqrt(mean_squared_error(testY[0], testPredict_lstm[:,0]))
print('Test Score: %.2f RMSE' % (testScore_lstm))

Train Score: 36.88 RMSE
Test Score: 41.10 RMSE


In [31]:
# shift train predictions for plotting
trainPredictPlot_lstm = np.empty_like(dataset)
trainPredictPlot_lstm[:, :] = np.nan
trainPredictPlot_lstm[look_back:len(trainPredict_lstm) + look_back, :] = trainPredict_lstm

In [33]:
# shift test predictions for plotting
testPredictPlot_lstm = np.empty_like(dataset)
testPredictPlot_lstm[:, :] = np.nan
testPredictPlot_lstm[len(trainPredict_lstm)+(look_back*2)+1:len(dataset)-2, :] = testPredict_lstm

In [34]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig.add_trace(go.Scatter(y=trainPredictPlot_lstm.flatten(), mode='lines', name='Train Predictions'))
fig.add_trace(go.Scatter(y=testPredictPlot_lstm.flatten(), mode='lines', name='Test Predictions'))

# Update layout for dark theme
fig.update_layout(title='LSTM Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
years = pd.to_datetime(df['time']).dt.year
unique_years = sorted(years.unique())
fig.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig.write_image('../figures/8_LSTM_preds_' + point_name + '.png')
fig.show()

### 5. Test CNN

Let's test a convolutional neural network (CNN) to compare to the LSTM.

In [37]:
point_name = 'middle' # 'upstream', 'middle', or 'terminus'
input_file='../data/ai_ready/' + point_name + '_timeseries.csv'

# load the dataset, drop the series column
df = pd.read_csv(input_file)

df.drop(columns="series", inplace=True)
dataset=df['velocity'].values.reshape(-1, 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets, 70% test data, 30% training data
train_size = int(len(dataset) * 0.70)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = trainX[:-1]
trainY = trainY[:-1]

# reshape input to be [samples, time steps, features]
trainX_cnn = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX_cnn = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

# create and fit the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(1))
cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.fit(trainX_cnn, trainY, epochs=500, batch_size=32, validation_data=(testX_cnn, testY), verbose=2)

# make predictions
trainPredict_cnn = cnn_model.predict(trainX_cnn)
testPredict_cnn = cnn_model.predict(testX_cnn)

# invert predictions
trainPredict_cnn = scaler.inverse_transform(trainPredict_cnn)
testPredict_cnn = scaler.inverse_transform(testPredict_cnn)

# calculate root mean squared error
trainScore_cnn = math.sqrt(mean_squared_error(trainY, trainPredict_cnn[:,0]))
print('Train Score: %.2f RMSE' % (trainScore_cnn))
testScore_cnn = math.sqrt(mean_squared_error(testY, testPredict_cnn[:,0]))
print('Test Score: %.2f RMSE' % (testScore_cnn))
# shift train predictions for plotting
trainPredictPlot_cnn = np.empty_like(dataset)
trainPredictPlot_cnn[:, :] = np.nan
trainPredictPlot_cnn[look_back:len(trainPredict_cnn) + look_back, :] = trainPredict_cnn

# shift test predictions for plotting
testPredictPlot_cnn = np.empty_like(dataset)
testPredictPlot_cnn[:, :] = np.nan
testPredictPlot_cnn[len(trainPredict_cnn) + (look_back * 2) + 1:len(dataset) - 2, :] = testPredict_cnn

# Create traces
fig_cnn = go.Figure()
fig_cnn.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig_cnn.add_trace(go.Scatter(y=trainPredictPlot_cnn.flatten(), mode='lines', name='Train Predictions (CNN)'))
fig_cnn.add_trace(go.Scatter(y=testPredictPlot_cnn.flatten(), mode='lines', name='Test Predictions (CNN)'))

# Update layout for dark theme
fig_cnn.update_layout(title='CNN Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
years = pd.to_datetime(df['time']).dt.year

unique_years = sorted(years.unique())

fig_cnn.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig_cnn.write_image('../figures/8_CNN_preds_' + point_name + '.png')
fig_cnn.show()

248 248
104 104
Epoch 1/500


/Users/cjense/miniconda3/envs/mlgeo-jensencc/lib/python3.10/site-packages/keras/layers/convolutional/base_conv.py:99: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 - 1s - 72ms/step - loss: 0.2662 - val_loss: 0.2124
Epoch 2/500
8/8 - 0s - 6ms/step - loss: 0.1011 - val_loss: 0.0631
Epoch 3/500
8/8 - 0s - 4ms/step - loss: 0.0293 - val_loss: 0.0115
Epoch 4/500
8/8 - 0s - 4ms/step - loss: 0.0246 - val_loss: 0.0135
Epoch 5/500
8/8 - 0s - 4ms/step - loss: 0.0245 - val_loss: 0.0111
Epoch 6/500
8/8 - 0s - 4ms/step - loss: 0.0192 - val_loss: 0.0158
Epoch 7/500
8/8 - 0s - 4ms/step - loss: 0.0194 - val_loss: 0.0160
Epoch 8/500
8/8 - 0s - 4ms/step - loss: 0.0192 - val_loss: 0.0131
Epoch 9/500
8/8 - 0s - 3ms/step - loss: 0.0188 - val_loss: 0.0119
Epoch 10/500
8/8 - 0s - 4ms/step - loss: 0.0187 - val_loss: 0.0125
Epoch 11/500
8/8 - 0s - 4ms/step - loss: 0.0186 - val_loss: 0.0129
Epoch 12/500
8/8 - 0s - 4ms/step - loss: 0.0185 - val_loss: 0.0132
Epoch 13/500
8/8 - 0s - 4ms/step - loss: 0.0177 - val_loss: 0.0126
Epoch 14/500
8/8 - 0s - 3ms/step - loss: 0.0180 - val_loss: 0.0119
Epoch 15/500
8/8 - 0s - 3ms/step - loss: 0.0181 - val_loss: 0.0121
Epoch 16/500
8/

In [38]:
# Print RMSE scores for both models
print('LSTM Train Score: %.2f RMSE' % (trainScore_lstm))
print('LSTM Test Score: %.2f RMSE' % (testScore_lstm))
print('CNN Train Score: %.2f RMSE' % (trainScore_cnn))
print('CNN Test Score: %.2f RMSE' % (testScore_cnn))

# Plot predictions of both models
fig_comparison = go.Figure()
fig_comparison.add_trace(go.Scatter(y=scaler.inverse_transform(dataset).flatten(), mode='lines', name='Actual Data'))
fig_comparison.add_trace(go.Scatter(y=trainPredictPlot_lstm.flatten(), mode='lines', name='Train Predictions (LSTM)'))
fig_comparison.add_trace(go.Scatter(y=testPredictPlot_lstm.flatten(), mode='lines', name='Test Predictions (LSTM)'))
fig_comparison.add_trace(go.Scatter(y=trainPredictPlot_cnn.flatten(), mode='lines', name='Train Predictions (CNN)'))
fig_comparison.add_trace(go.Scatter(y=testPredictPlot_cnn.flatten(), mode='lines', name='Test Predictions (CNN)'))

# Update layout for dark theme
fig_comparison.update_layout(title='LSTM vs CNN Predictions vs Actual Data for ' + point_name.capitalize() + ' Point', xaxis_title='Time', yaxis_title='Velocity')
# Update x-axis to show only one tick per year
fig_comparison.update_xaxes(tickvals=[df.index[years == year][0] for year in unique_years], ticktext=unique_years)
fig_comparison.show()

LSTM Train Score: 36.88 RMSE
LSTM Test Score: 41.10 RMSE
CNN Train Score: 821.66 RMSE
CNN Test Score: 843.57 RMSE
